In [2]:
pip install scipy


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import os
import sys
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pickle
import scipy
from sklearn.model_selection import train_test_split
import seaborn as sns



from openpyxl import Workbook

In [5]:
fn = r'train_test_sets/combdata_080224.xlsx'
# sheet_name = "all data"
df = pd.read_excel(fn,header=0)
df.columns = [ent.strip() for ent in df.columns]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop(columns=['Unnamed: 0'])
df.to_excel(fn)

# Convert a dataset with many electrolyte columns into 6 conc and smiles columns

In [280]:
def shift_values(row, conc_columns, smiles_columns):
    # Extract concentration and corresponding smiles values
    conc_values = [row[col] for col in conc_columns]
    smiles_values = [row[col] for col in smiles_columns]

    # Create a list of tuples (conc, smiles)
    combined = list(zip(conc_values, smiles_values))

    # Filter out the zero concentration values
    non_zero = [(conc, smiles) for conc, smiles in combined if conc != 0]
    zero = [(0, None)] * (len(conc_values) - len(non_zero))

    # Combine non-zero values followed by zero values
    new_combined = non_zero + zero

    # Separate the combined list back into conc_values and smiles_values
    conc_values, smiles_values = zip(*new_combined)

    # Update the row with the new values
    for i, col in enumerate(conc_columns):
        row[col] = conc_values[i]
    for i, col in enumerate(smiles_columns):
        row[col] = smiles_values[i]
    
    return row

def remove_zero_columns(df, conc_columns, smiles_columns):
    # Keep only the first two conc and smiles columns
    keep_conc = ['conc1', 'conc2','conc3','conc4','conc5','conc6']
    keep_smiles = ['smiles1', 'smiles2','smiles3','smiles4','smiles5','smiles6']

    for i, row in df.iterrows():
        for conc_col, smiles_col in zip(conc_columns, smiles_columns):
            if conc_col not in keep_conc and row[conc_col] == 0:
                df.at[i, smiles_col] = None
                df.at[i, conc_col] = None

    # Drop columns where all values are None
    df.dropna(axis=1, how='all', inplace=True)

    return df



In [ ]:
smiles_columns = [col for col in df.columns if 'smiles' in col]
conc_columns = [col for col in df.columns if 'conc' in col]
# col_set = [conc_cols,smiles_cols]

# Strip whitespace from the strings in both DataFrames
for col in smiles_columns:
    df[col] = df[col].str.strip()

shift_data = df.apply(shift_values,axis=1, args=(conc_columns,smiles_columns))
shift_data

# Replace empty strings with NaN
shift_data[smiles_columns] = shift_data[smiles_columns].replace('', np.nan)

# Fill NaN values with 'O'
shift_data[smiles_columns] = shift_data[smiles_columns].fillna('O')
data2 = remove_zero_columns(shift_data, conc_columns, smiles_columns)

# Remove rows with empty values in 'k (mS/cm)' column
data2 = data2.dropna(subset=['k (mS/cm)'])
data2 = data2[data2['k (mS/cm)'] != 0]

# # Remove the excess smiles and conc_columns
# for i in range(7,len(smiles_columns)+1):
#     data2.drop(columns=f'conc{i}')
#     data2.drop(columns=f'smiles{i}')

# data2.to_excel("J13_all.xlsx")
df2 = data2
df2

In [299]:
fn_smile = r"/Users/murtazazohair/Library/CloudStorage/Box-Box/IBM INTERNAL - Battery Team Database_2019-2024/AI-ML Projects/Ionic Conductivity/SMILES Materials Info.xlsx"
smiles_df = pd.read_excel(fn_smile,header=0,usecols=[0,1,2,3,4,5,6]) 


smiles_df['SMILES'] = smiles_df['SMILES'].str.strip()
# density_dict = smiles_data.set_index('SMILES')['Specific Gravity (g/mL)'].to_dict()
# molar_mass_dict = smiles_data.set_index('SMILES')['MW (mol/g)'].to_dict()


In [300]:
def conversions(row,smiles_df): 
    
    for i in range(2, 7):
    # Apply density multiplication and save as new mass columns

        smiles_col = f'smiles{i}'
        conc_col = f'conc{i}'
        vol_col = f'vol{i}'
        mass_col = f'mass{i}'
        mol_col = f'mol{i}'
        smile = row[smiles_col]
        if smile in smiles_df['SMILES'].values:
            density = smiles_df.loc[smiles_df['SMILES'] == smile, 'Specific Gravity (g/mL)'].values[0]
            molar_mass = smiles_df.loc[smiles_df['SMILES'] == smile, 'MW (mol/g)'].values[0]
        else:
            print('error in smiles:' + smile)
            break
        if row['solvent ratio type'] == 'v':
                row[vol_col] = row[conc_col]
                row[mass_col] = row[conc_col] * density
                row[mol_col] = row[mass_col] / molar_mass

        elif row['solvent ratio type'] == 'w':
            row[mass_col] = row[conc_col]
            row[vol_col] = row[mass_col] / density
            row[mol_col] = row[mass_col] / molar_mass
       
        elif row['solvent ratio type'] == 'mol':
            row[mol_col] = row[conc_col]
            row[mass_col] = row[mol_col] * molar_mass
            row[vol_col] = row[mass_col] / density
        
        else:
            print('error in conversions')
            break

    if row['c units'] == 'mol/l':
        total_vol = sum(row[f'vol{i}'] for i in range(2, 7))
        row['mol1'] = row['conc1'] * total_vol/1000
    elif row['c units'] == 'mol/kg':
        total_mass = sum(row[f'mass{i}'] for i in range(2, 7))
        row['mol1'] = row['conc1'] * total_mass/1000    


    # Calculate molp columns
    total_mol = sum(row[f'mol{i}'] for i in range(1, 7))
    for i in range(1, 7):
        mol_col = f'mol{i}'
        molp_col = f'molp{i}'
        row[molp_col] = row[mol_col] / total_mol*100 if total_mol != 0 else 0
    
    return row

        



In [303]:
# Apply the operations
df_new = df2
df_new = df_new.apply(conversions, axis=1,smiles_df=smiles_df)

# Drop mol, vol, conc, and mass columns
cols_to_drop = [f'vol{i}' for i in range(2, 7)] + [f'mass{i}' for i in range(2, 7)] + [f'mol{i}' for i in range(1, 7)] + [f'conc{i}' for i in range(1, 7)]
df_new.drop(columns=cols_to_drop, inplace=True)

# Rename molp columns to conc columns
rename_dict = {f'molp{i}': f'conc{i}' for i in range(1, 7)}
df_new.rename(columns=rename_dict, inplace=True)

# df_new.to_excel('J13_converted.xlsx')

In [ ]:
# Combine conc, smiles, and temps columns for duplicate checking
conc_cols = [f'conc{i}' for i in range(1, 7)]
smiles_cols = [f'smiles{i}' for i in range(1, 7)]
combined_cols = conc_cols + smiles_cols + ['T']

# Check for duplicate sets of values across combined columns
duplicate_sets = df_new[combined_cols].duplicated(keep=False)
duplicates_df = df_new[duplicate_sets]

# List to store duplicate sets
duplicate_sets_list = duplicates_df[combined_cols].drop_duplicates()

# Prepare to store cleaned DataFrame
df_cleaned = df_new[~duplicate_sets]

# Process each duplicate set
n = 0
for _, group in duplicates_df.groupby(combined_cols):
    # Calculate the average of 'k (mS/cm)' for the current duplicate set
    avg_k_value = group['k (mS/cm)'].mean()
    
    # Create a new row with the averaged 'k (mS/cm)' value
    new_row = group.iloc[0].copy()  # Start with one of the duplicate rows
    new_row['k (mS/cm)'] = avg_k_value
    
    # Append the new row to the cleaned DataFrame
    df_cleaned = df_cleaned.append(new_row, ignore_index=True)
    n += 1

# Convert Temperature from K to C
df_cleaned['temperature ( C )'] = df_cleaned['T'] - 273.15

# Convert k to log cond
df_cleaned['log(cond) (uS/cm)'] = np.log10(df_cleaned['k (mS/cm)']*1000)


# Output duplicate sets
# print("Duplicate sets:")
duplicate_sets_list.to_excel('J13_duplicates.xlsx')

# # Output cleaned DataFrame
# print("\nCleaned DataFrame with duplicates averaged:")
df_cleaned.to_excel('J13_converted.xlsx')
df_cleaned

# Filter Dataset for Source

In [6]:
filt_df = df[(df['source'] != "MZ_J4") & (df['source'] != "MZ_J8")]
filt_df


,source,temperature ( C ),smiles1,smiles2,smiles3,smiles4,smiles5,smiles6,conc1,conc2,conc3,conc4,conc5,conc6,log(cond) (uS/cm)
0,MZ_J11,20.00,COCCOC,[Li+].[N+](=O)([O-])[O-],O,O,O,O,96.912765,3.087235,0.000000,0.000000,0.000000,0,2.301030
1,MZ_J11,20.00,COCCOC,[Li+].[B-]1(OC(=O)C(=O)O1)(F)F,O,O,O,O,94.958373,5.041627,0.000000,0.000000,0.000000,0,3.591065
2,MZ_J11,20.00,COCCOC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,O,O,O,O,94.958373,5.041627,0.000000,0.000000,0.000000,0,3.707570
3,MZ_J11,20.00,COCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,O,O,O,O,94.958373,5.041627,0.000000,0.000000,0.000000,0,3.963788
4,MZ_J11,20.00,COCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,[Li+].[B-]1(OC(=O)C(=O)O1)(F)F,[Li+].[N+](=O)([O-])[O-],O,94.958373,1.260407,1.260407,1.260407,1.260407,0,4.082785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13732,MZ_J13,10.00,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,15.177120,25.851870,58.971010,0.000000,0.000000,0,3.848098
13733,MZ_J13,20.00,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,15.177120,25.851870,58.971010,0.000000,0.000000,0,3.971551
13734,MZ_J13,30.00,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,15.177120,25.851870,58.971010,0.000000,0.000000,0,4.071132
13735,MZ_J13,40.00,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,15.177120,25.851870,58.971010,0.000000,0.000000,0,4.144753


# Remove outlier values of IC

In [9]:
max(filt_df['temperature ( C )'])

204.2734

# Filter Dataset with Temperatures

In [4]:
filt_df = df[(df['temperature ( C )'] >= 10) & (df['temperature ( C )'] <= 30)]
filt_df

,source,temperature ( C ),smiles1,smiles2,smiles3,smiles4,smiles5,smiles6,conc1,conc2,conc3,conc4,conc5,conc6,log(cond) (uS/cm),cond (mS/cm)
0,MZ_J11,20.00,COCCOC,[Li+].[N+](=O)([O-])[O-],O,O,O,O,96.912765,3.087235,0.000000,0.000000,0.000000,0,2.301030,0.200000
1,MZ_J11,20.00,COCCOC,[Li+].[B-]1(OC(=O)C(=O)O1)(F)F,O,O,O,O,94.958373,5.041627,0.000000,0.000000,0.000000,0,3.591065,3.900000
2,MZ_J11,20.00,COCCOC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,O,O,O,O,94.958373,5.041627,0.000000,0.000000,0.000000,0,3.707570,5.100000
3,MZ_J11,20.00,COCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,O,O,O,O,94.958373,5.041627,0.000000,0.000000,0.000000,0,3.963788,9.200000
4,MZ_J11,20.00,COCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,[Li+].[B-]1(OC(=O)C(=O)O1)(F)F,[Li+].[N+](=O)([O-])[O-],O,94.958373,1.260407,1.260407,1.260407,1.260407,0,4.082785,12.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18764,MZ_J13,30.00,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,11.831770,26.871450,61.296780,0.000000,0.000000,0,4.110091,12.885195
18767,MZ_J13,19.85,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,CC1COC(=O)O1,COC(=O)OC,O,O,14.969980,27.097730,7.864249,50.068030,0.000000,0,3.792944,6.207890
18770,MZ_J13,10.00,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,15.177120,25.851870,58.971010,0.000000,0.000000,0,3.848098,7.048521
18771,MZ_J13,20.00,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,15.177120,25.851870,58.971010,0.000000,0.000000,0,3.971551,9.365932


Split data from J4 and select 200 random points to add back to the dataset

In [339]:
# J4 = filt_df[filt_df['source'] == ('MZ_J4')]
filt_df = df[df['source'] != "MZ_J4"]
# filt_data_list = []
# for i in range(1):
#     sample = J4.sample(n=200, replace=False)
#     J4_drop = J4.drop(sample.index)
#     filt_data_list.append(pd.concat([filt_data2,sample],ignore_index=True))
# filt_data_list
filt_df
filt_df.to_excel("combdata_080224.xlsx")

Create 80/20 train/test datasets

In [5]:
# property_columns = ["log(cond) (uS/cm)"]
property_columns = ["cond (mS/cm)"]

formulations_concer_columns = ["conc1", "conc2", "conc3", "conc4","conc5","conc6"]
formulations_smiles_columns = ["smiles1", "smiles2", "smiles3", "smiles4","smiles5","smiles6"]
temperature = ["temperature ( C )"]

# n = 1
# for data in filt_data_list:

n = 'combdata_072924_RT_2'
X = filt_df[formulations_concer_columns+formulations_smiles_columns+temperature]
Y = filt_df[property_columns]

train_features, test_features, train_labels, test_labels = train_test_split(X, Y,test_size=0.2,random_state=42) 
train_data = train_labels.join(train_features)
test_data = test_labels.join(test_features)
train_data.to_csv(f'train_test_sets/train_{n}.csv')
test_data.to_csv(f'train_test_sets/test_{n}.csv')

    # n += 1


In [6]:
train_data

,cond (mS/cm),conc1,conc2,conc3,conc4,conc5,conc6,smiles1,smiles2,smiles3,smiles4,smiles5,smiles6,temperature ( C )
17543,11.999993,11.879400,8.993206,79.127400,0.000000,0.0,0,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,COC(=O)OC,O,O,O,30.0
17542,10.149999,11.903350,88.096650,0.000000,0.000000,0.0,0,[Li+].F[P-](F)(F)(F)(F)F,COC(=O)OC,O,O,O,O,30.0
223,2.987232,18.223259,10.007421,24.170465,47.598855,0.0,0,[Li+].F[P-](F)(F)(F)(F)F,C1=COC(=O)O1,C1COC(=O)O1,CCOC(=O)OC,O,O,10.0
12,2.158742,22.222222,55.555556,22.222222,0.000000,0.0,0,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,C1CCS(=O)(=O)C1,C(C(C(F)F)(F)F)OC(C(F)F)(F)F,O,O,O,20.0
14152,5.178786,4.893941,60.341520,34.764530,0.000000,0.0,0,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,CC1COC(=O)O1,CCOC(=O)OCC,O,O,O,29.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18376,9.810000,6.387132,70.015550,23.597320,0.000000,0.0,0,[Li+].F[As-](F)(F)(F)(F)F,C(Cl)Cl,C1CCOC1,O,O,O,20.0
2753,6.445630,31.611246,15.286162,46.124716,6.977875,0.0,0,[Li+].F[P-](F)(F)(F)(F)F,C1COC(=O)O1,CCOC(=O)OC,CC1COC(=O)O1,O,O,10.0
12005,6.314966,10.489460,38.980480,50.530060,0.000000,0.0,0,[Li+].F[P-](F)(F)(F)(F)F,CC1COC(=O)O1,CCOC(=O)OCC,O,O,O,19.5
15602,2.141702,1.160736,43.043000,55.796270,0.000000,0.0,0,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,CC1COC(=O)O1,CCOC(=O)OCC,O,O,O,29.4


In [25]:
new_data = pd.concat([data_0C,data_10C,data_20C,data_30C,data_40C,data_50C,data_60C])
new_data
new_data.to_excel(r"/Users/murtazazohair/Library/CloudStorage/Box-Box/IBM INTERNAL - Battery Team Database_2019-2023/AI-ML Projects/Ionic Conductivity/data.xlsx")